# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [ ]:
# reference architecture https://www.geeksforgeeks.org/implement-your-own-word2vecskip-gram-model-in-python/

In [ ]:
# Imports
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import torch
import tensorflow as tf
from numpy import array
import numpy as np
from numpy import argmax
import torch.nn as nn


In [ ]:
torch.cuda.is_available()
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"

# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [ ]:
#TODO: implement!

#from google.colab import files
#uploaded = files.upload()

data = pd.read_table('hindi_dataset.tsv',nrows=100)
data

#loading data from the tsv

## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [ ]:
#TODO: implement!
data['text']= data['text'].str.lower()
data['text'] = data['text'].str.replace('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]','')

file1 = open("stopwords.txt", encoding="utf8") 
q=file1.read()
split1=q.split()

data['text'] = data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in split1]))

data

#remove the punctuations and stopwords

## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [ ]:
#TODO: implement!
# V = [w for w in hindi_tokens if not w in hindi_stop_words]
# print(V)

V = data['text'].values.tolist()
V= [word for line in V for word in line.split()]

print(V)

In [ ]:
freq = {} 
relative_freq={}
p_keep={}
for item in V: 
    if (item in freq): 
        freq[item] += 1
    else: 
        freq[item] = 1


for item in freq:
    relative_freq[item]=freq[item]/len(V)
    
        
        
for x, y in relative_freq.items():
    p_keep[x]=(np.sqrt(y/0.001) +1) * (0.001/y) 



print(p_keep)

* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [ ]:
def word_to_one_hot(word):
    one_hot_vector=np.zeros((len(freq)))
    
    index=list(freq.keys()).index(word)
    one_hot_vector[index]=1
    
    return one_hot_vector
       


#---test data--#
#corresponding_one_hot_encoded=word_to_one_hot('बांग्लादेश')

#print(corresponding_one_hot_encoded)

In [ ]:
#to send the variable freq so that it can be accessed in task2

file = open("hindi_freq_dictionary.txt", "wb")
pickle.dump(freq, file)
file.close()

In [ ]:
def sampling_prob(word):
    return p_keep[word]


#--test--#
#print(sampling_prob('रखना'))
#----

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [ ]:
# Set hyperparameters
window_size = 5
embedding_size = 300


In [ ]:
def get_target_context_helper(variable_sentence,current_index):
    #takes in the new sentence and the curresponding index
    
    word_list=[]
    i=current_index
    j=current_index
    lower_limit=current_index-window_size
    upper_limit=current_index+window_size

    if lower_limit<0:
        lower_limit=0
    if upper_limit> len(variable_sentence)-1:
        upper_limit=len(variable_sentence)-1

    while(i>lower_limit):
        word_list.append(variable_sentence[i-1])
        i=i-1

    while (j<upper_limit):
        word_list.append(variable_sentence[j+1])
        j=j+1
    
    return variable_sentence[current_index], word_list
    #returns the currespong word and the list of words in its context

def get_target_context(sentence):
    
    sentence=sentence.split()
    new_sentence=[]
    for each_word in sentence:
        float_value=float(p_keep[each_word])
        if np.random.random() < float_value:
            new_sentence.append(each_word)
    
    p=[]
   
    len_new=len(new_sentence)
    for i in range (len_new):
        #print(get_target_context_helper(new_sentence,i)) #passes the new sentences and each index to do the window
        p.append(get_target_context_helper(new_sentence,i))

        
    return p



each_sentence=data['text']
list_of_all_words_and_corresponding_context_words=[]
for every_sentence in each_sentence:
    list_of_all_words_and_corresponding_context_words.append(get_target_context(every_sentence))
    

list_of_all_words_and_corresponding_context_words = [item for sublist in list_of_all_words_and_corresponding_context_words for item in sublist]
print(list_of_all_words_and_corresponding_context_words)
#returns the list of all the focus and context words for the entire vocabulary

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [ ]:
# More hyperparameters
learning_rate = 0.025
epochs = 100

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [ ]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self,embedding_size,Vocabulary_size):
    super(Word2Vec,self).__init__()
    
    self.l1=nn.Linear(Vocabulary_size,embedding_size)
    self.l2=nn.Linear(embedding_size,Vocabulary_size)
    self.softmax=nn.Softmax(dim=1)


  def forward(self, one_hot):
    out=self.l1(one_hot)
    out=self.l2(out)
    out=self.softmax(out)
    return out


model=Word2Vec(embedding_size,len(freq))



In [ ]:
#onehot to torch transform

def transform_to_torch(corresponding_one_hot_encoded):
    corresponding_one_hot_encoded_torch = torch.from_numpy(corresponding_one_hot_encoded.astype(np.float32))
    corresponding_one_hot_encoded_torch = corresponding_one_hot_encoded_torch.view(corresponding_one_hot_encoded_torch.shape[0],1)
    corresponding_one_hot_encoded_torch=corresponding_one_hot_encoded_torch.t()
    return corresponding_one_hot_encoded_torch

# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [ ]:
# Define optimizer and loss
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = torch.nn.NLLLoss()

    

# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [ ]:
# Define train procedure

# load initial weights

#for each focus word all the corresponding context words are fed into thr model and trained
n_iters=100

def train():
    for epoch in range(n_iters):
        
        for each_word in list_of_all_words_and_corresponding_context_words:
            focus,context_words=each_word
            #print("focus is",focus)
            
            focus =word_to_one_hot(focus)
            
            focus =transform_to_torch(focus)
            #print(focus.shape)
            
            for each_context_word in context_words:
                #print("contexts are",each_context_word)
                each_context_word=word_to_one_hot(each_context_word)
                each_context_word =transform_to_torch(each_context_word)
                Y_pred=model(focus)
                #torch.max(labels, 1)[1]
                loss=criterion(Y_pred,torch.max(each_context_word, 1)[1])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
        print("loss is",loss)

#train() #uncomment the train function to run for 100 epochs.

print("Training finished")

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).

In [ ]:
#save the model as task1.pth
file_name = F"task1.pth"
torch.save(model.state_dict(), file_name)

#for implementation representation the data was run on 100 rows. The code for the entire data setwas run in collab and saved
#task1.pth